In [1]:
def claent(dataset):
    n = dataset.shape[0]
    iset = dataset.iloc[:,-1].value_counts()
    p = iset/n
    ent = (-p*np.log2(p)).sum()
    return ent

In [2]:
import numpy as np
import pandas as pd
def createDataset():
    raw = {"no_surfacing":[1,1,1,0,0],
          "flippers":[1,1,0,1,1],
          "fish":["yes","yes","no","no","no"]}
    dataset = pd.DataFrame(raw)
    return dataset

In [5]:
dataset = createDataset()
dataset

,no_surfacing,flippers,fish
0,1,1,yes
1,1,1,yes
2,1,0,no
3,0,1,no
4,0,1,no


In [7]:
claent(dataset)

0.9709505944546686

In [10]:
def bestsplit(dataset):
    baseEnt = claent(dataset)
    bestGain = 0
    axis = -1
    for i in range(dataset.shape[1]-1):
        levels = dataset.iloc[:,i].value_counts().index
        ents = 0
        for j in levels:
            childset = dataset[dataset.iloc[:,i] ==j]
            ent = claent(childset)
            ents+= (childset.shape[0]/dataset.shape[0])*ent
        print(f"第{i}列的信息熵为{ents}")
        infoGain = baseEnt - ents
        print(f"第{i}列的信息增益为{infoGain}")
        if (infoGain > bestGain):
            bestGain = infoGain
            axis = i
    return axis
            

In [11]:
bestsplit(dataset)

第0列的信息熵为0.5509775004326937
第0列的信息增益为0.4199730940219749
第1列的信息熵为0.8
第1列的信息增益为0.17095059445466854


0

In [12]:
def mysplit(dataset ,axis ,value):
    col = dataset.columns[axis]
    redataset = dataset.loc[dataset[col] == value,:].drop(col,axis = 1)
    return redataset

In [13]:
mysplit(dataset,0,1)

,flippers,fish
0,1,yes
1,1,yes
2,0,no


In [20]:
def createTree(dataset):
    featlist = list(dataset.columns)
    classlist = dataset.iloc[:,-1].value_counts()
    if classlist[0] == dataset.shape[0] or dataset.shape[1] ==1:
        return classlist.index[0]
    axis = bestsplit(dataset)
    bestfeat = featlist[axis]
    myTree = {bestfeat:{}}
    del featlist[axis]
    valuelist = set(dataset.iloc[:,axis])
    for value in valuelist:
        myTree[bestfeat][value] = createTree(mysplit(dataset,axis,value))
    return myTree

In [21]:
myTree = createTree(dataset)
myTree

第0列的信息熵为0.5509775004326937
第0列的信息增益为0.4199730940219749
第1列的信息熵为0.8
第1列的信息增益为0.17095059445466854
第0列的信息熵为0.0
第0列的信息增益为0.9182958340544896


{'no_surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [39]:
def classify(inputTree,labels,testvec):
    firststr = next(iter(inputTree))
    seconddist = inputTree[firststr]
    featindex = labels.index(firststr)
    for key in seconddist.keys():
        if testvec[featindex] == key:
            if type(seconddist[key]) == dict :
                classlabel = classify(seconddist[key] , labels ,testvec)
            else:
                classlabel = seconddist[key]
    return classlabel

In [44]:
firststr = next(iter(myTree))

In [45]:
firststr

'no_surfacing'

In [25]:
(iter(myTree))

In [42]:
labels = list(dataset.columns)
labels

['no_surfacing', 'flippers', 'fish']

In [29]:
labels = dataset.iloc[:,-1].unique().tolist()

In [46]:
labels.index(firststr)

0

In [40]:
def acc_classify(train,test):
    inputTree = createTree(train) #根据训练集生成一棵树
    labels = list(train.columns) #数据集所有的列名称
    result = []
    for i in range(test.shape[0]):
        testVec = test.iloc[i,:-1] #测试集中的一个实例
        classLabel = classify(inputTree,labels,testVec) #预测该实例的分类
        result.append(classLabel)
    test['predict'] = result
    acc = (test.iloc[:,-1] == test.iloc[:,-2]).mean() #计算准确度
    print(f'模型预测准确率为{acc}')
    return test
 
 
if __name__ == '__main__':
    dataSet = createDataset()
    train = dataSet
    #用训练集的前三个作为测试样本
    test = dataSet.iloc[:3,:]
    print(acc_classify(train,test))

第0列的信息熵为0.5509775004326937
第0列的信息增益为0.4199730940219749
第1列的信息熵为0.8
第1列的信息增益为0.17095059445466854
第0列的信息熵为0.0
第0列的信息增益为0.9182958340544896
模型预测准确率为1.0
   no_surfacing  flippers fish predict
0             1         1  yes     yes
1             1         1  yes     yes
2             1         0   no      no


E:\environment\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [41]:
#使用sklearn中graphviz包实现决策树的绘制
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
 
#特征
xtrain = dataSet.iloc[:,:-1]
#标签
ytrain = dataSet.iloc[:,-1]
labels = ytrain.unique().tolist()
ytrain = ytrain.apply(lambda x:labels.index(x))
#绘制树模型
clf = DecisionTreeClassifier()
clf = clf.fit(xtrain,ytrain)
tree.export_graphviz(clf)

'digraph Tree {\nnode [shape=box] ;\n0 [label="X[0] <= 0.5\\ngini = 0.48\\nsamples = 5\\nvalue = [2, 3]"] ;\n1 [label="gini = 0.0\\nsamples = 2\\nvalue = [0, 2]"] ;\n0 -> 1 [labeldistance=2.5, labelangle=45, headlabel="True"] ;\n2 [label="X[1] <= 0.5\\ngini = 0.444\\nsamples = 3\\nvalue = [2, 1]"] ;\n0 -> 2 [labeldistance=2.5, labelangle=-45, headlabel="False"] ;\n3 [label="gini = 0.0\\nsamples = 1\\nvalue = [0, 1]"] ;\n2 -> 3 ;\n4 [label="gini = 0.0\\nsamples = 2\\nvalue = [2, 0]"] ;\n2 -> 4 ;\n}'